# Week 5 - Vector Space Model (VSM) and Topic Modeling

Over the next weeks, we are going to re-implement Sherin's algorithm and apply it to the text data we've been working on last week! Here's our roadmap:

**Week 6 - vectorization and linear algebra**
6. Dampen: weight the frequency of words (1 + log[count])
7. Scale: Normalize weighted frequency of words
8. Direction: compute deviation vectors

**Week 7 - Clustering**
9. apply different unsupervised machine learning algorithms
    * figure out how many clusters we want to keep
    * inspect the results of the clustering algorithm

**Week 8 - Visualizing the results**
10. create visualizations to compare documents

# WEEK 5 - DATA CLEANING

## Step 1 - Data Retrieval

In [45]:
# using glob, find all the text files in the "Papers" folder
import glob

files = glob.glob('../week5-vsm-1-ghcsung/Papers/*.txt')
print(files)

['../week5-vsm-1-ghcsung/Papers\\paper0.txt', '../week5-vsm-1-ghcsung/Papers\\paper1.txt', '../week5-vsm-1-ghcsung/Papers\\paper10.txt', '../week5-vsm-1-ghcsung/Papers\\paper11.txt', '../week5-vsm-1-ghcsung/Papers\\paper12.txt', '../week5-vsm-1-ghcsung/Papers\\paper13.txt', '../week5-vsm-1-ghcsung/Papers\\paper14.txt', '../week5-vsm-1-ghcsung/Papers\\paper15.txt', '../week5-vsm-1-ghcsung/Papers\\paper16.txt', '../week5-vsm-1-ghcsung/Papers\\paper2.txt', '../week5-vsm-1-ghcsung/Papers\\paper3.txt', '../week5-vsm-1-ghcsung/Papers\\paper4.txt', '../week5-vsm-1-ghcsung/Papers\\paper5.txt', '../week5-vsm-1-ghcsung/Papers\\paper6.txt', '../week5-vsm-1-ghcsung/Papers\\paper7.txt', '../week5-vsm-1-ghcsung/Papers\\paper8.txt', '../week5-vsm-1-ghcsung/Papers\\paper9.txt']


In [46]:
# get all the data from the text files into the "documents" list
# P.S. make sure you use the 'utf-8' encoding
documents = []

for filename in files: 
    with open (filename, "r", encoding='utf-8') as f:
        documents.append(f.read())

In [47]:
# print the first 1000 characters of the first document to see what it 
# looks like (we'll use this as a sanity check below)
documents[0][:1500]

'\x0czone out no more: mitigating mind wandering during\ncomputerized reading\nsidney k. d’mello, caitlin mills, robert bixler, & nigel bosch\nuniversity of notre dame\n118 haggar hall\nnotre dame, in 46556, usa\nsdmello@nd.edu\n\nabstract\nmind wandering, defined as shifts in attention from task-related\nprocessing to task-unrelated thoughts, is a ubiquitous\nphenomenon that has a negative influence on performance and\nproductivity in many contexts, including learning. we propose\nthat next-generation learning technologies should have some\nmechanism to detect and respond to mind wandering in real-time.\ntowards this end, we developed a technology that automatically\ndetects mind wandering from eye-gaze during learning from\ninstructional texts. when mind wandering is detected, the\ntechnology intervenes by posing just-in-time questions and\nencouraging re-reading as needed. after multiple rounds of\niterative refinement, we summatively compared the technology to\na yoked-control in a

## Step 2 - Data Cleaning

In [48]:
# only select the text that's between the first occurence of the 
# the word "abstract" and the last occurence of the word "reference"
# Optional: print the length of the string before and after, as a 
# sanity check
# HINT: https://stackoverflow.com/questions/14496006/finding-last-occurrence-of-substring-in-string-replacing-that
# read more about rfind: https://www.tutorialspoint.com/python/string_rfind.htm

for i,doc in enumerate(documents):
    print(len(documents[i]), end=' ')
    # only keep the text after the abstract
    doc = doc[doc.index('abstract'):doc.rfind('reference')]
    # save the result
    documents[i] = doc
    # print the length of the resulting string
    print(len(documents[i]))
    
# one liner:
# documents = [doc[doc.index('abstract'):doc.rfind('reference')] for doc in documents]

50043 39318
41110 35514
49177 42621
32277 28206
40387 34778
45258 42251
40655 32734
31574 28134
42046 37649
46761 42253
47377 42978
44037 40032
37214 32762
47851 41302
42617 35102
45724 39947
47845 44059


In [49]:
# replace carriage returns (i.e., "\n") with a white space
# check that the result looks okay by printing the 
# first 1000 characters of the 1st doc:

documents = [doc.replace('\n', ' ') for doc in documents]
print(documents[0][:1000])

abstract mind wandering, defined as shifts in attention from task-related processing to task-unrelated thoughts, is a ubiquitous phenomenon that has a negative influence on performance and productivity in many contexts, including learning. we propose that next-generation learning technologies should have some mechanism to detect and respond to mind wandering in real-time. towards this end, we developed a technology that automatically detects mind wandering from eye-gaze during learning from instructional texts. when mind wandering is detected, the technology intervenes by posing just-in-time questions and encouraging re-reading as needed. after multiple rounds of iterative refinement, we summatively compared the technology to a yoked-control in an experiment with 104 participants. the key dependent variable was performance on a post-reading comprehension assessment. our results suggest that the technology was successful in correcting comprehension deficits attributed to mind wandering 

In [6]:
# replace the punctation below by a white space
# check that the result looks okay 
# (e.g., by print the first 1000 characters of the 1st doc)

punctuation = ['.', '...', '!', '#', '"', '%', '$', "'", '&', ')', 
               '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', 
               '<', '?', '>', '@', '",', '".', '[', ']', '\\', ',',
               '_', '^', '`', '{', '}', '|', '~', '−', '”', '“', '’']


# remove ponctuation
for i,doc in enumerate(documents): 
    for punc in punctuation: 
        doc = doc.replace(punc, ' ')
    documents[i] = doc
    
print(documents[0][:1000])

abstract mind wandering  defined as shifts in attention from task related processing to task unrelated thoughts  is a ubiquitous phenomenon that has a negative influence on performance and productivity in many contexts  including learning  we propose that next generation learning technologies should have some mechanism to detect and respond to mind wandering in real time  towards this end  we developed a technology that automatically detects mind wandering from eye gaze during learning from instructional texts  when mind wandering is detected  the technology intervenes by posing just in time questions and encouraging re reading as needed  after multiple rounds of iterative refinement  we summatively compared the technology to a yoked control in an experiment with 104 participants  the key dependent variable was performance on a post reading comprehension assessment  our results suggest that the technology was successful in correcting comprehension deficits attributed to mind wandering 

In [50]:
# remove numbers by either a white space or the word "number"
# again, print the first 1000 characters of the first document
# to check that you're doing the right thing
for i,doc in enumerate(documents): 
    for num in range(10):
        doc = doc.replace(str(num), '')
    documents[i] = doc

print(documents[1][:1000])

abstract educational systems typically contain a large pool of items (questions, problems). using data mining techniques we can group these items into knowledge components, detect duplicated items and outliers, and identify missing items. to these ends, it is useful to analyze item similarities, which can be used as input to clustering or visualization techniques. we describe and evaluate different measures of item similarity that are based only on learners’ performance data, which makes them widely applicable. we provide evaluation using both simulated data and real data from several educational systems. the results show that pearson correlation is a suitable similarity measure and that response times are useful for improving stability of similarity measures when the scope of available data is small.  . introduction interactive educational systems offer learners items (problems, questions) for solving. realistic educational systems typically contain a large number of such items. this 

In [51]:
# Remove the stop words below from our documents
# print the first 1000 characters of the first document
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
              'ourselves', 'you', 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his', 'himself', 'she', 
              'her', 'hers', 'herself', 'it', 'its', 'itself', 
              'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 
              'be', 'been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
              'but', 'if', 'or', 'because', 'as', 'until', 'while', 
              'of', 'at', 'by', 'for', 'with', 'about', 'against', 
              'between', 'into', 'through', 'during', 'before', 
              'after', 'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 'again', 
              'further', 'then', 'once', 'here', 'there', 'when', 
              'where', 'why', 'how', 'all', 'any', 'both', 'each', 
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 
              'nor', 'not', 'only', 'own', 'same', 'so', 'than', 
              'too', 'very', 's', 't', 'can', 'will', 
              'just', 'don', 'should', 'now']


# remove stop words
for i,doc in enumerate(documents):
    for stop_word in stop_words:
        doc = doc.replace(' ' + stop_word + ' ', ' ')
    documents[i] = doc

print(documents[0][:1000])

abstract mind wandering, defined shifts attention task-related processing task-unrelated thoughts, ubiquitous phenomenon negative influence performance productivity many contexts, including learning. propose next-generation learning technologies mechanism detect respond mind wandering real-time. towards end, developed technology automatically detects mind wandering eye-gaze learning instructional texts. mind wandering detected, technology intervenes posing just-in-time questions encouraging re-reading needed. multiple rounds iterative refinement, summatively compared technology yoked-control experiment  participants. key dependent variable performance post-reading comprehension assessment. results suggest technology successful correcting comprehension deficits attributed mind wandering (d = . sigma) specific conditions, thereby highlighting potential improve learning “attending attention.”  keywords mind wandering; gaze tracking; student modeling; attentionaware.  . introduction despit

In [52]:
# remove words with one and two characters (e.g., 'd', 'er', etc.)
# print the first 1000 characters of the first document

for i,doc in enumerate(documents):  
    doc = [x for x in doc.split() if len(x) > 2]
    doc = " ".join(doc)
    documents[i] = doc

print(documents[0][:1000])

abstract mind wandering, defined shifts attention task-related processing task-unrelated thoughts, ubiquitous phenomenon negative influence performance productivity many contexts, including learning. propose next-generation learning technologies mechanism detect respond mind wandering real-time. towards end, developed technology automatically detects mind wandering eye-gaze learning instructional texts. mind wandering detected, technology intervenes posing just-in-time questions encouraging re-reading needed. multiple rounds iterative refinement, summatively compared technology yoked-control experiment participants. key dependent variable performance post-reading comprehension assessment. results suggest technology successful correcting comprehension deficits attributed mind wandering sigma) specific conditions, thereby highlighting potential improve learning “attending attention.” keywords mind wandering; gaze tracking; student modeling; attentionaware. introduction despite best effor


### Putting it all together

In [10]:
# package all of your work above into a function that cleans a given document

def clean_list_of_documents(documents):
    
    cleaned_docs = []

    for i,doc in enumerate(documents):
        # only keep the text after the abstract
        doc = doc[doc.index('abstract'):]
        # only keep the text before the references
        doc = doc[:doc.rfind('reference')]
        # replace return carriage with white space
        doc = doc.replace('\n', ' ')
        # remove ponctuation
        for punc in punctuation: 
            doc = doc.replace(punc, ' ')
        # remove numbers
        for i in range(10):
            doc = doc.replace(str(i), ' ')
        # remove stop words
        for stop_word in stop_words:
            doc = doc.replace(' ' + stop_word + ' ', ' ')
        # remove single characters and stem the words 
        doc = [x for x in doc.split() if len(x) > 2]
        doc = " ".join(doc)
        # save the result to our list of documents
        cleaned_docs.append(doc)
        
    return cleaned_docs

In [53]:
# reimport your raw data
documents = []

for filename in files: 
    with open (filename, "r", encoding='utf-8') as f:
        documents.append(f.read())
        
# clean your files using the function above
docs = clean_list_of_documents(documents)

# print the first 1000 characters of the first document
print(docs[0][:1000])

abstract mind wandering defined shifts attention task related processing task unrelated thoughts ubiquitous phenomenon negative influence performance productivity many contexts including learning propose next generation learning technologies mechanism detect respond mind wandering real time towards end developed technology automatically detects mind wandering eye gaze learning instructional texts mind wandering detected technology intervenes posing time questions encouraging reading needed multiple rounds iterative refinement summatively compared technology yoked control experiment participants key dependent variable performance post reading comprehension assessment results suggest technology successful correcting comprehension deficits attributed mind wandering sigma specific conditions thereby highlighting potential improve learning attending attention keywords mind wandering gaze tracking student modeling attentionaware introduction despite best efforts write clear engaging paper ch

## Step 3 - Build your list of vocabulary

This list of words (i.e., the vocabulary) is going to become the columns of your matrix.

In [19]:
import math
import numpy as np

In [54]:
# create a function that takes in a list of documents
# and returns a set of unique words. Make sure that you
# sort the list alphabetically before returning it. 

def get_vocabulary(docs):
    voc = []
    for doc in docs:
        for word in doc.split():
            if word not in voc: 
                voc.append(word)
    voc = list(set(voc))
    voc.sort()
    return voc

# Then print the length of your vocabulary (it should be 
# around 5500 words)
vocabulary = get_vocabulary(docs)
print(len(vocabulary))

5676


## Step 4 - transform your documents in to 100-words chunks

In [55]:
# 15) create a function that takes in a list of documents
# and returns a list of 100-words chunk 
# (with a 25 words overlap between them)
# Optional: add two arguments, one for the number of words
# in each chunk, and one for the overlap size
# Advice: combining all the documents into one giant string
# and splitting it into separate words will make your life easier!

def flatten_and_overlap(docs, window_size=100, overlap=25):
    
    # create the list of overlapping documents
    new_list_of_documents = []
    
    # flatten everything into one string
    flat = ""
    for doc in docs:
        flat += doc
    
    # split into words
    flat = flat.split()

    # create chunks of 100 words
    high = window_size
    while high < len(flat):
        low = high - window_size
        new_list_of_documents.append(flat[low:high])
        high += overlap
    return new_list_of_documents

chunks = flatten_and_overlap(docs)


# def split_to_chunk(doc, length, overlap): 
#     # join documents into bigstring, split into words
#     bigstring = ''
#     for i in doc:
#         bigstring += i
#     words = bigstring.split(' ')
    
#     # create list of num-sized chunks with overlap
#     chunks = []
#     for i in range(0,len(words)-length+1, length - overlap):
#         #print('len(words-length+1: ', len(words)-length+1, ' length-overlap: ',length-overlap)
#         chunk = words[i:i+length]
#         #print(len(chunk))
#         chunks.append(chunk)
#     return chunks
        
# #print(len(split_to_chunk(doc, 100, 25)))
# chunked = split_to_chunk(doc,100,25)
# chunked[0]

In [56]:
# create a for loop to double check that each chunk has 
# a length of 100
# Optional: use assert to do this check
for chunk in chunks: 
    assert(len(chunk) == 100)
print(chunks[0])

['abstract', 'mind', 'wandering', 'defined', 'shifts', 'attention', 'task', 'related', 'processing', 'task', 'unrelated', 'thoughts', 'ubiquitous', 'phenomenon', 'negative', 'influence', 'performance', 'productivity', 'many', 'contexts', 'including', 'learning', 'propose', 'next', 'generation', 'learning', 'technologies', 'mechanism', 'detect', 'respond', 'mind', 'wandering', 'real', 'time', 'towards', 'end', 'developed', 'technology', 'automatically', 'detects', 'mind', 'wandering', 'eye', 'gaze', 'learning', 'instructional', 'texts', 'mind', 'wandering', 'detected', 'technology', 'intervenes', 'posing', 'time', 'questions', 'encouraging', 'reading', 'needed', 'multiple', 'rounds', 'iterative', 'refinement', 'summatively', 'compared', 'technology', 'yoked', 'control', 'experiment', 'participants', 'key', 'dependent', 'variable', 'performance', 'post', 'reading', 'comprehension', 'assessment', 'results', 'suggest', 'technology', 'successful', 'correcting', 'comprehension', 'deficits', 

# WEEK 6 - VECTOR MANIPULATION

## Step 5 - Create a word by document matrix

In [86]:
# 1) create an empty dataframe using pandas
# the number of rows should be the number of documents we have
# the number of columns should be size of the vocabulary
import pandas as pd
df = pd.DataFrame(0,index = range(len(chunks)), columns = vocabulary)

In [83]:
# len(vocabulary)
# len(chunks)
df.columns
df_test = df.from_dict(dict.fromkeys(chunks[0], [0]))
for word in chunks[0]:
    if word in list(df.columns):
        df_test.loc[0,word] += 1
print(df_test)


   abstract  assessment  attending  attention  attributed  automatically  \
0         1           1          1          2           1              1   

   compared  comprehension  conditions  contexts  ...    texts  thereby  \
0         1              2           1         1  ...        1        1   

   thoughts  time  towards  ubiquitous  unrelated  variable  wandering  yoked  
0         1     2        1           1          1         1          6      1  

[1 rows x 78 columns]


In [87]:
# 2) fill out the dataframe with the count of words for each document
# (use two for loops to iterate through the documents and the vocabulary)
for i, chunk in enumerate(chunks):
    for word in chunk:
        if word in list(df.columns):
        #print(word)
            df.loc[i, word] += 1
        #count = chunk.count(word) ];
        #df.loc[i, word] = count
    if i % 100 == 0:
        print(i, end=' ')
        

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 

In [99]:
# 3) Sanity check: make sure that your counts are correct
# (e.g., if you know that a words appears often in a document, check that
# the number is also high in your dataframe; and vice-versa for low counts)
sum(df.learning)


2455

In [96]:
# 4) Putting it together: create a function that takes a list of documents
# and a vocabulary as arguments, and returns a dataframe with the counts
# of words: 
def create_wordfreq_df(documents, vocabulary):
    import pandas as pd
    df = pd.DataFrame(0,index=range(len(documents)), columns = range(len(vocabulary)))
    for i, chunk in enumerate(chunks):
        for word in chunk:
            if word in list(df.columns):
                df.loc[i, word] += 1
    if i % 100 == 0:
        print(i, end=' ')
        
# call the function and check that the resulting dataframe is correct
result = create_wordfreq_df(chunks, vocabulary)

KeyboardInterrupt: 

## Step 6 - Weight word frequency

In [97]:
# 5) create a function that adds one to the current cell and takes its log
# IF the value in the cell is not zero

def add1_log(x):
    import numpy as np
    if x != 0:
        return np.log(x+1)
    else:
        return 0


In [100]:
# 6) use the "applymap" function of the dataframe to apply the function 
# above to each cell of the table
# df = df.from_csv('./word-by-chunk.csv', encoding= 'utf-8')

df_weigh = df.applymap(lambda x: add1_log(x))


0       0.000000
1       0.693147
2       0.693147
3       0.693147
4       0.693147
5       0.000000
6       0.000000
7       0.000000
8       0.000000
9       0.000000
10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.000000
20      0.000000
21      0.000000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
          ...   
2189    0.693147
2190    0.000000
2191    0.000000
2192    0.000000
2193    0.000000
2194    0.000000
2195    0.000000
2196    0.000000
2197    0.000000
2198    0.000000
2199    0.000000
2200    0.000000
2201    0.000000
2202    0.000000
2203    0.000000
2204    0.000000
2205    0.693147
2206    1.098612
2207    1.386294
2208    1.386294
2209    1.386294
2210    1.098612
2211    0.693147
2212    0.693147
2213    0.000000
2214    0.693147
2215    0.693147
2216    0.6931

In [ ]:
# 7) check that the numbers in the resulting matrix look accurate;
# print the value before and after applying the function above
df['student']
df_weigh['student']


## Step 7 - Matrix normalization

In [ ]:
# 8) look at the image below; why do you think that we need to normalize our 
# data before clustering in this particular case? 

# The scales for the two variables are completely different. One ranges from -4 to 4, the other from -3000 to 4000.
# If we clustered this data without normalizing it, the latter feature will overpower any varience in the former. 

<img src="https://i.stack.imgur.com/N2unM.png" />

In general, it's common practice to normalize your data before clustering - so that variables are comparable.

In [ ]:
# 9) describe how the min-max normalization works:
# The minimum value is converted to zero, the maximum value to one, 
# and the values falling between the two scaled depending on where it is in regards to the minimum and maximum values. 

<img src="https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/media/aml-normalization-minmax.png" />

In [ ]:
# 10) describe how normalizing using a z-score works:
# This normalizes the distribution of the data to a normal distribution, with mean = 0 and standard deviation = 1.

<img src="https://cdn-images-1.medium.com/max/1600/1*13XKCXQc7eabfZbRzkvGvA.gif"/>

In [37]:
# 11) describe how normalizing to unit norm works
# This scales the vectors, which in this context is the different features of each datapoint, to have a length of one.
# To do this, every entry in the vector is divided by its magnitude. 


1
2
3


Resources: 
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer
* http://mathworld.wolfram.com/NormalVector.html

We are going to work with some pre-made normalization functions from sklearn (feel free to skim this page):
* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

In [126]:
# 12) since we are working with vectors, apply the Normalizer from 
# sklearn.preprocessing to our dataframe. Print a few values 
# before and after to make sure you've applied the normalization
from sklearn.preprocessing import Normalizer
print(df_weigh['student'])
a = list(df_weigh.columns).index('student')
transformer = Normalizer().fit(df_weigh)
norm_np = pd.DataFrame.from_records(transformer.transform(df_weigh))
norm_np[a]
print(norm_np.shape)


0       0.000000
1       0.693147
2       0.693147
3       0.693147
4       0.693147
5       0.000000
6       0.000000
7       0.000000
8       0.000000
9       0.000000
10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.000000
20      0.000000
21      0.000000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
          ...   
2189    0.693147
2190    0.000000
2191    0.000000
2192    0.000000
2193    0.000000
2194    0.000000
2195    0.000000
2196    0.000000
2197    0.000000
2198    0.000000
2199    0.000000
2200    0.000000
2201    0.000000
2202    0.000000
2203    0.000000
2204    0.000000
2205    0.693147
2206    1.098612
2207    1.386294
2208    1.386294
2209    1.386294
2210    1.098612
2211    0.693147
2212    0.693147
2213    0.000000
2214    0.693147
2215    0.693147
2216    0.6931

In [38]:
# 13) create a function that takes a dataframe as argument and where a second
# argument is the type of normalization (MinMaxScaler, Normalizer, StandardScaler)
# and returns the normalized dataframe
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

def normalize_choice(df, norm):
    if norm == 'MinMaxScaler':
        scaler = StandardScaler()
        return scaler.transform(df)
    elif norm == 'Normalizer':
        scaler = Normalizer().fit(df)
        return scaler.transform(df)
    elif norm == 'StandardScaler':
        scaler = StandardScaler()
        return scaler.transform(df)
    else:
        print('Input type of normalization: MinMaxScaler, Normalizer, or StandardScaler')

# the output is a numpy array? So is the input for the deviation vector a numpy array?
# Do we change this back into a dataframe at some point? If so, where and how?


## Step 8 - Deviation Vectors

<img src="https://www.dropbox.com/s/9f73r7pk7bi7vh9/deviation_vectors.png?dl=1" />

In [39]:
# 14) compute the sum of the vectors
# I'm using df to practice here, because the outputs of the previous function is a numpy array...
# Will have to add a line to convert when we put the functions together. 
sum_vec = df.sum(axis = 1)
print(len(df.column))

2219


In [44]:
# 15) normalize the vector (find its average)
avrg_vec = sum_vec/len(df.columns)


In [125]:
# 16) take each vector and subtract its components along v_avg
dev_vec = df - avrg_vec

In [ ]:
# 17) put the code above in a function that takes in a dataframe as an argument
# and computes deviation vectors of each row (=document)
def calc_deviation_vec(df):
    sum_vec = df.sum(axis = 1)
    avrg_vec = sum_vec/len(df.columns)
    return (df - avrg_vec)

# WEEK 7 - CLUSTERING

## Step 9 - Clustering

## Step 10 - Visualizing the results

## Final Step - Putting it all together: 

In [ ]:
# in python code, our goal is to recreate the steps above as functions
# so that we can just one line to run topic modeling on a list of 
# documents: 
def ExtractTopicsVSM(documents, numTopics):
    ''' this functions takes in a list of documents (strings), 
        runs topic modeling (as implemented by Sherin, 2013)
        and returns the clustering results, the matrix used 
        for clustering a visualization '''
    
    # step 2: clean up the documents
    documents = clean_list_of_documents(documents)
    
    # step 3: let's build the vocabulary of these docs
    vocabulary = get_vocabulary(documents)
    
    # step 4: we build our list of 100-words overlapping fragments
    documents = flatten_and_overlap(documents)
    
    # step 5: we convert the chunks into a matrix
    matrix = docs_by_words_matrix(documents, vocabulary)
    
    # step 6: we weight the frequency of words (count = 1 + log(count))
    matrix = one_plus_log_mat(matrix, documents, vocabulary)
    
    # step 7: we normalize the matrix
    matrix = normalize(matrix)
    
    # step 8: we compute deviation vectors
    matrix = transform_deviation_vectors(matrix, documents)
    
    # step 9: we apply a clustering algorithm to find topics
    results_clustering = cluster_matrix(matrix)
    
    # step 10: we create a visualization of the topics
    visualization = visualize_clusters(results_clustering, vocabulary)
    
    # finally, we return the clustering results, the matrix, and a visualization
    return results_clustering, matrix, visualization